# Setup

In [ ]:
import os
import joblib

In [ ]:
do_sas = False
do_stata = False
do_ABB = False
do_LRT = False
do_ConSav = False

# Functions to call SAS, Stata and MATLAB

In [ ]:
def run_sas(file):
    
    # a. run
    sasexe = '"C:/Program Files/SASHome/SASFoundation/9.4/sas.exe"'
    out = os.system(f'{sasexe} -sysin {os.getcwd()}/{file}.sas -ICON -NOSPLASH')
    
    # b. check if terminated succesfully
    if out == 0:
        print(f'{file} terminated succesfully')
    else:
        raise Exception(f'{file} did not terminate succesfully')    
        
    # c. clean up
    os.remove(f'{file}.log')        

In [ ]:
def run_stata(file):
    
    # a. run
    stataexe = '"C:\Program Files (x86)\Stata15\StataMP-64.exe"'
    os.system(f'{stataexe} /UseRegistryStartin /e {os.getcwd()}/{file}.do')
    
    # b. check if terminated succesfully
    with open(f'{file}.log') as logfile:
        lastline = list(logfile)[-1]
        if lastline == 'end of do-file\n':
            print(f'{file} terminated succesfully')
        else:
            raise Exception(f'{file} did not terminate succesfully\n {lastline}')
    
    # c. clean up
    os.remove(f'{file}.log')

In [ ]:
def run_matlab(file):
    
    # a. run
    out = os.system('matlab -nosplash -nodesktop -wait -r "run(' + "'" + f'{os.getcwd()}/{file}' + ".m');exit;")
    
    # b. check if terminated succesfully
    if out == 0:
        print(f'{file} terminated succesfully')
    else:
        raise Exception(f'{file} did not terminate succesfully')    

In [ ]:
def run_jupyter(file):
    
    os.system(f'jupyter nbconvert --to notebook --execute --ExecutePreprocessor.timeout=-1 --inplace "{file}.ipynb"')
    

In [ ]:
def run_jupyter_parallel(files):
    
    tasks = (joblib.delayed(run_jupyter)(ipynbfile) for ipynbfile,do in ipynbfiles if do)
    joblib.Parallel(n_jobs=len(files))(tasks)


# Fetch data and select sample(s)

**Inputs:**

1. Danish register data (INDUPD, BEF, FAIN, IDAP, with views located in H:/Rawdata/706248/views/)
* Consumer price data: data/PRIS61.xlsx (from Statistikbanken.dk)
* House price data: data/EJEN6.xlsx (from Statistikbanken.dk)

## Run SAS

In [ ]:
# a. setup
sasfiles = [('0_1_import_datasets',True),
            ('0_2_merge_datasets',True),
            ('0_3_export_datasets',True)]

# b. run
if do_sas:
    [run_sas(sasfile) for sasfile,do in sasfiles if do]

**Output:** data/sample_raw.dta

## Run STATA

In [ ]:
# a. setup
statafiles = [('1_1_construct_deflator',True),
              ('1_2_construct_deflator_house',True),
              ('1_3_deflate',True),
              ('1_4_export_csv',True)]

# b. run
if do_stata:
    [run_stata(statafile) for statafile,do in statafiles if do]

**Output:**

1. data/logY_SAMPLE.csv
* data/logY_raw_SAMPLE.csv
* data/birthyear_SAMPLE.csv
* data/a_mean_SAMPLE.csv
* data/a_p50_SAMPLE.csv
* data/Y_zerocutoff_SAMPLE.csv
* data/logY_unbalanced_SAMPLE.csv

**Results:**

1. output/units.xls
* output/totobs.xls
* output/units_zerocutoff.xls
* output/totobs_zerocutoff.xls
* output/units_unbalanced.xls
* output/totobs_unbalanced.xls

# Arellano-Blundell-Bonhomme

In [ ]:
# a. setup
mfiles = [('ABB/run',True)]

# b. run
if do_ABB:
    [run_matlab(mfile) for mfile,do in mfiles if do]

**Output:** data/abb_sim.csv

# LRT

In [ ]:
# a. setup
ipynbfiles = [('2_1_main_results',True),
              ('2_2_cross_validation',True),
              ('2_3_attrition',True),
              ('2_4_unbalanced',True),
              ('2_5_levels',True),
              ('2_6_cohorts',True)]

# b. run
if do_LRT:
    run_jupyter_parallel(ipynbfiles)

**Output:**

1. data/mean_logY.txt.txt
* data/mean_Y.txt
* data/mean_Y_lev.txt
* data/num_leafs.txt
* data/prob_G_ini.txt
* data/var_logY.txt
* data/ypred_G_t*.txt
* data/tran_t*.txt

**Results:**

1. output/PT_estimates.txt
2. output/*.pdf

# Consumption-saving model

In [ ]:
# a. setup
mfiles = [('ConSav/run',True)]

# b. run
if do_ConSav:
    [run_matlab(mfile) for mfile,do in mfiles if do]

**Results:**

1. output/ConSav/main_table.tex
* output/ConSav/*.pdf
* output/ConSavEstimates/*.mat